In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F 
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import math
import copy
import nbimporter

In [25]:
df = pd.read_csv(r'D:\PROGRAMMING\BE proj24\dataset\angles.csv')
df.head()

,vid_id,frame_order,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder
0,0,0,16.926802,7.667874,18.982162,112.747505,112.62553,112.099300,101.05565
1,0,1,14.199318,8.954973,18.966124,109.707190,109.76263,110.645454,102.00027
2,0,2,18.065800,10.315741,17.527954,114.562100,112.08965,113.340350,104.09502
3,0,3,23.270214,17.336140,17.195545,117.674810,115.43172,114.634530,107.38297
4,0,4,22.831680,13.822096,17.355429,117.536720,117.96766,112.306390,98.39078


DATA PRE-PROCESSING

In [26]:
squats_df=df[(df['vid_id']>=385)& (df['vid_id']<=447)]
squats_df.head(11817)

,vid_id,frame_order,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder
72105,385,0,53.402206,73.528010,46.317870,149.35954,153.83890,118.31155,88.452630
72106,385,1,45.957460,81.406560,35.316982,118.72570,145.04758,108.92687,110.066660
72107,385,2,44.079243,79.323600,37.196740,103.62296,137.77670,103.73133,104.628716
72108,385,3,34.982906,78.514984,41.724506,86.89696,123.90274,93.21554,105.458960
72109,385,4,30.291750,78.184020,39.902603,87.05850,124.05390,87.08308,101.913830
...,...,...,...,...,...,...,...,...,...
83917,447,296,107.305360,90.115130,51.515130,134.34811,147.88246,136.57085,123.315970
83918,447,297,100.225420,94.343120,53.821750,146.84540,163.03261,140.31600,140.187360
83919,447,298,100.277790,89.232956,49.927440,137.95894,152.91435,136.71500,128.460190
83920,447,299,99.322820,87.257810,49.641770,148.99538,167.36041,138.74146,134.618380


In [27]:
scaler = MinMaxScaler()
squats_df.iloc[:, 2:] = scaler.fit_transform(squats_df.iloc[:, 2:])
squats_df.head()

C:\Users\siddh\AppData\Local\Temp\ipykernel_14604\3053181253.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  squats_df.iloc[:, 2:] = scaler.fit_transform(squats_df.iloc[:, 2:])


,vid_id,frame_order,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder
72105,385,0,0.295393,0.411265,0.273035,0.829341,0.849218,0.658835,0.485592
72106,385,1,0.253775,0.455628,0.204535,0.654619,0.798166,0.606105,0.608375
72107,385,2,0.243276,0.443899,0.216240,0.568480,0.755944,0.576912,0.577484
72108,385,3,0.192425,0.439346,0.244433,0.473082,0.675377,0.517826,0.582200
72109,385,4,0.166201,0.437482,0.233089,0.474003,0.676255,0.483369,0.562061


In [28]:
#MISSING VALUES
squats_df.isna().sum()

vid_id                                    0
frame_order                               0
right_elbow_right_shoulder_right_hip      0
left_elbow_left_shoulder_left_hip         0
right_knee_mid_hip_left_knee              0
right_hip_right_knee_right_ankle          0
left_hip_left_knee_left_ankle             0
right_wrist_right_elbow_right_shoulder    0
left_wrist_left_elbow_left_shoulder       0
dtype: int64

In [29]:
batches = {vid_id: batch_df for vid_id, batch_df in squats_df.groupby('vid_id')}
print(batches[386])
batches[386].shape

       vid_id  frame_order  right_elbow_right_shoulder_right_hip  \
72297     386            0                              0.312283   
72298     386            1                              0.339067   
72299     386            2                              0.285885   
72300     386            3                              0.244346   
72301     386            4                              0.238285   
...       ...          ...                                   ...   
72410     386          113                              0.423775   
72411     386          114                              0.302198   
72412     386          115                              0.378614   
72413     386          116                              0.412370   
72414     386          117                              0.441989   

       left_elbow_left_shoulder_left_hip  right_knee_mid_hip_left_knee  \
72297                           0.534412                      0.336942   
72298                           0.5

(118, 9)

In [30]:
len(batches)

63

In [31]:
squats_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
vid_id,11817.0,417.327664,17.565843,385.0,402.000000,418.000000,432.000000,447.0
frame_order,11817.0,116.823644,75.504366,0.0,54.000000,109.000000,171.000000,300.0
right_elbow_right_shoulder_right_hip,11817.0,0.409176,0.212706,0.0,0.252715,0.404604,0.549629,1.0
left_elbow_left_shoulder_left_hip,11817.0,0.410467,0.201786,0.0,0.256526,0.401572,0.543662,1.0
right_knee_mid_hip_left_knee,11817.0,0.363427,0.206590,0.0,0.190267,0.323593,0.505297,1.0
right_hip_right_knee_right_ankle,11817.0,0.628201,0.255368,0.0,0.389640,0.688924,0.856644,1.0
left_hip_left_knee_left_ankle,11817.0,0.616586,0.255721,0.0,0.385659,0.666495,0.845015,1.0
right_wrist_right_elbow_right_shoulder,11817.0,0.716137,0.158534,0.0,0.632566,0.731869,0.828845,1.0
left_wrist_left_elbow_left_shoulder,11817.0,0.695448,0.172723,0.0,0.598088,0.716101,0.825546,1.0


In [32]:
sample_vid_id = 385
sample_batch = squats_df['vid_id']
sample_batch

72105    385
72106    385
72107    385
72108    385
72109    385
        ... 
83917    447
83918    447
83919    447
83920    447
83921    447
Name: vid_id, Length: 11817, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf


angle_columns = ['left_hip_left_knee_left_ankle', 'right_hip_right_knee_right_ankle']

for vid_id, batch in batches.items():
    print(f"PACF vid_id: {vid_id}")
    num=len(batch)
    value_lags=num//2
    for angle in angle_columns:
        angle_data = batch[angle].dropna().values
        
        plt.figure(figsize=(10, 6))
        plot_pacf(angle_data, lags=value_lags, method='ywm')
        plt.title(f'PACF for {angle} (vid_id {vid_id}) - No Rolling Mean')
        plt.xlabel('Lag')
        plt.ylabel('Partial Autocorrelation')
        plt.show()

In [33]:
rolling_window = 5

# Preprocess each batch: apply rolling mean, then normalize
preprocessed_batches = {}

for vid_id, batch in batches.items():
    print(f"video ID: {vid_id}")
    print(" batch before rolling mean:")
    print(batch.head())
    angle_columns = batch.columns[2:]
    batch[angle_columns] = batch[angle_columns].rolling(window=rolling_window, min_periods=0).mean()
    print("batch after applying rolling mean:")
    print(batch.head())
   
    # Store the preprocessed batch
    preprocessed_batches[vid_id] = batch

# Display the first batch to verify preprocessing
preprocessed_batches[385].head()


video ID: 385
 batch before rolling mean:
       vid_id  frame_order  right_elbow_right_shoulder_right_hip  \
72105     385            0                              0.295393   
72106     385            1                              0.253775   
72107     385            2                              0.243276   
72108     385            3                              0.192425   
72109     385            4                              0.166201   

       left_elbow_left_shoulder_left_hip  right_knee_mid_hip_left_knee  \
72105                           0.411265                      0.273035   
72106                           0.455628                      0.204535   
72107                           0.443899                      0.216240   
72108                           0.439346                      0.244433   
72109                           0.437482                      0.233089   

       right_hip_right_knee_right_ankle  left_hip_left_knee_left_ankle  \
72105                         

,vid_id,frame_order,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder
72105,385,0,0.295393,0.411265,0.273035,0.829341,0.849218,0.658835,0.485592
72106,385,1,0.274584,0.433446,0.238785,0.741980,0.823692,0.632470,0.546984
72107,385,2,0.264148,0.436930,0.231270,0.684147,0.801110,0.613951,0.557150
72108,385,3,0.246217,0.437534,0.234561,0.631381,0.769676,0.589919,0.563413
72109,385,4,0.230214,0.437524,0.234267,0.599905,0.750992,0.568609,0.563143


In [ ]:
angle_columns = ['left_hip_left_knee_left_ankle', 'right_hip_right_knee_right_ankle']
for vid_id, batch in preprocessed_batches.items():
    print(f"PACF for vid_id: {vid_id} with rolling mean")
    num=len(batch)
    value_lags=num//2
    for angle in angle_columns:
        angle_data = batch[angle].dropna().values  # Remove any NaN values
        
        plt.figure(figsize=(10, 6))
        plot_acf(angle_data, lags=value_lags)
        plt.title(f'ACF for {angle} (vid_id {vid_id}) with Rolling Mean')
        plt.xlabel('Lag')
        plt.ylabel('Partial Autocorrelation')
        plt.show()

        plt.figure(figsize=(10, 6))
        plot_pacf(angle_data, lags=value_lags)
        plt.title(f'PACF for {angle} (vid_id {vid_id}) with Rolling Mean')
        plt.xlabel('Lag')
        plt.ylabel('Partial Autocorrelation')
        plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Sample video ID and batch
sample_vid_id = 385
sample_batch = preprocessed_batches[sample_vid_id]

# Angle columns to visualize
angle_columns = ['left_hip_left_knee_left_ankle', 'right_hip_right_knee_right_ankle']

# Create a figure with a larger size to avoid overlap
plt.figure(figsize=(12, 8))

# Loop through each angle column
for i, angle in enumerate(angle_columns):
    # Get the original data and the rolling mean (this was already calculated)
    original_data = squats_df[squats_df['vid_id'] == sample_vid_id][angle]
    rolling_mean = sample_batch[angle]  # This is the rolling mean from preprocessing

    # Create a subplot for each angle column
    plt.subplot(len(angle_columns), 1, i + 1)  # len(angle_columns) rows, 1 column

    # Plot the original data (raw data) - in blue, solid line
    plt.plot(sample_batch['frame_order'], original_data, label=f'Original {angle}', color='blue', linewidth=2)
    
    # Plot the rolling mean (smoothed data) - in red, dashed line
    plt.plot(sample_batch['frame_order'], rolling_mean, label=f'Rolling Mean {angle}', color='red', linestyle='--')

    # Add titles and labels
    plt.title(f'Raw vs Rolling Mean for {angle} (vid_id {sample_vid_id})')
    plt.xlabel('Frame Order')
    plt.ylabel(f'{angle} (degrees)')
    plt.legend()

# Adjust layout to avoid overlap and ensure spacing is correct
plt.tight_layout()
plt.show()


In [34]:
unbatched_data = pd.concat(preprocessed_batches.values())
unbatched_data


,vid_id,frame_order,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder
72105,385,0,0.295393,0.411265,0.273035,0.829341,0.849218,0.658835,0.485592
72106,385,1,0.274584,0.433446,0.238785,0.741980,0.823692,0.632470,0.546984
72107,385,2,0.264148,0.436930,0.231270,0.684147,0.801110,0.613951,0.557150
72108,385,3,0.246217,0.437534,0.234561,0.631381,0.769676,0.589919,0.563413
72109,385,4,0.230214,0.437524,0.234267,0.599905,0.750992,0.568609,0.563143
...,...,...,...,...,...,...,...,...,...
83917,447,296,0.603747,0.503169,0.318448,0.683055,0.762868,0.748895,0.664447
83918,447,297,0.591808,0.506000,0.314108,0.716617,0.793298,0.760584,0.692144
83919,447,298,0.582774,0.517964,0.308398,0.741786,0.815432,0.765945,0.716099
83920,447,299,0.572767,0.512631,0.304648,0.764117,0.843678,0.768156,0.727172


In [35]:
unbatched_data.to_csv('squats_angles.csv', index=False)

In [36]:
loaded_data = pd.read_csv(r"D:\PROGRAMMING\New folder\Rehabilitation-System\squats_angles.csv")
loaded_data.head(11817)

,vid_id,frame_order,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder
0,385,0,0.295393,0.411265,0.273035,0.829341,0.849218,0.658835,0.485592
1,385,1,0.274584,0.433446,0.238785,0.741980,0.823692,0.632470,0.546984
2,385,2,0.264148,0.436930,0.231270,0.684147,0.801110,0.613951,0.557150
3,385,3,0.246217,0.437534,0.234561,0.631381,0.769676,0.589919,0.563413
4,385,4,0.230214,0.437524,0.234267,0.599905,0.750992,0.568609,0.563143
...,...,...,...,...,...,...,...,...,...
11812,447,296,0.603747,0.503169,0.318448,0.683055,0.762868,0.748895,0.664447
11813,447,297,0.591808,0.506000,0.314108,0.716617,0.793298,0.760584,0.692144
11814,447,298,0.582774,0.517964,0.308398,0.741786,0.815432,0.765945,0.716099
11815,447,299,0.572767,0.512631,0.304648,0.764117,0.843678,0.768156,0.727172


In [ ]:
# scaler = MinMaxScaler()
# preprocessed_batches = {}
# for vid_id, batch in batches.items():
    
#     # Normalize angle columns (excluding 'vid_id' and 'frame_order')
    
#     batch.iloc[:, 2:] = scaler.fit_transform(batch.iloc[:, 2:])
    
#     # Store the preprocessed batch
#     preprocessed_batches[vid_id] = batch

# # Show the first batch after preprocessing
# preprocessed_batches[386].head(118)


In [ ]:
A=squats_df.iloc[:,2:]
A

In [ ]:
class ExerciseDataset(Dataset):
    def __init__(self, data):
        # self.data = data
        # self.r_data = data[df.iloc[:,2:]].values
        self.data = data
        # Define the columns for X and y
        self.columns = [
            'right_elbow_right_shoulder_right_hip', 
            'left_elbow_left_shoulder_left_hip', 
            'right_knee_mid_hip_left_knee', 
            'right_hip_right_knee_right_ankle', 
            'left_hip_left_knee_left_ankle', 
            'right_wrist_right_elbow_right_shoulder', 
            'left_wrist_left_elbow_left_shoulder'
        ]
        # Extract only these columns from the DataFrame
        self.r_data = data[self.columns].values
    def __len__(self):
        # Since y is the next frame, we have one less sample in y
        return len(self.data) - 1

    def __getitem__(self, idx):
        # X is the current frame, y is the next frame in the sequence
        X = torch.tensor(self.r_data[idx], dtype=torch.float32)
        y = torch.tensor(self.r_data[idx + 1], dtype=torch.float32)  # Next frame as target
        return X, y



In [ ]:
total_rows = squats_df.shape[0]
BATCH_SIZE=64

# Define an 80-20 split
train_size = int(0.8 * total_rows)
train_videos = squats_df[:train_size]
test_videos = squats_df[train_size:]

# Check the size of the splits
len(train_videos), len(test_videos)


In [ ]:
# Pass the train and test DataFrames into the custom Dataset
train_dataset = ExerciseDataset(train_videos)
test_dataset = ExerciseDataset(test_videos)

# Create DataLoaders from the datasets
train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False)


In [ ]:
for i, batch in enumerate(train_dataloader):
    print(f"Batch {i}:")
    print(batch)
    if i == 0:  # Change 2 to the number of batches you'd like to preview
        break

In [ ]:
print(train_dataset)

In [ ]:
next(iter(train_dataloader))

In [ ]:
train_videos.shape

In [ ]:
test_videos.shape

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 7),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


In [ ]:
model = NeuralNetwork().to(device)
print(model)


In [ ]:
loss_fn = nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def r_squared(y, pred):
    # Calculate the total sum of squares
    ss_total = torch.sum((y - torch.mean(y)) ** 2)
    
    # Calculate the residual sum of squares
    ss_residual = torch.sum((y -pred) ** 2)
    
    # Calculate R-squared
    r2 = 1 - (ss_residual / ss_total)
    
    return r2

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
size = len(train_dataloader.dataset)
size

In [ ]:
iter(test_dataloader)

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for i in range(0, len(dataloader), BATCH_SIZE):
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                data=X
                targets =y
                pred = model(data)           
                total_loss += loss_fn(pred, torch.mean(y, dim=0).unsqueeze(0)).item()
                r2 = r_squared(y,pred)
            print(f"Total loss:\n R2 score: {r2}, Avg loss: {total_loss:>8f} \n")
    return total_loss / len(dataloader),r2
            


In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)



In [ ]:
# Save the model
model_path = 'data.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


In [ ]:
def predict_future(model, dataloader):
    model.eval()   
    with torch.no_grad():
        for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                data=X
                targets =y
                pred = model(data)
                print(pred)
                break
predict_future(model, test_dataloader)

In [ ]:
model = NeuralNetwork().to(device)  # Create a new instance of your model
model.load_state_dict(torch.load(model_path))  # Load the saved state dict
model.eval()  # Set the model to evaluation mode
print("Model loaded successfully.")

In [ ]:
# Define which batch and column to visualize
pred_batch_num = 1  # Index of batch to visualize (first batch in this case)
col_id = 6 # Column index to visualize within the data

dataloader_iter = iter(test_dataloader)


for _ in range(pred_batch_num):
    next(dataloader_iter)

# Now, retrieve the specified batch
X, y = next(dataloader_iter)
X, y = X.to(device), y.to(device)

# Make predictions using the model
pred = model(X)

# Move predictions and actual values back to CPU for plotting
pred = pred.cpu().detach().numpy()
y = y.cpu().detach().numpy()


# Visualize the specified column in the batch
plt.figure(figsize=(10, 6))

# Plot the actual target data for the specified column
plt.plot(y[:, col_id], label='Actual Data', color='blue')

# Plot the model's predictions for the same column
plt.plot(pred[:, col_id], label='Predictions', color='red')

# Additional plot settings
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.title('Model Predictions vs Actual Data for Column {}'.format(col_id))
plt.show()


In [ ]:
# Convert the train_dataloader to a list to access batches by index directly
train_dataloader_list = list(train_dataloader)

# Define the batch number to visualize
train_batch_num = 147  # Specify the batch number you want to visualize

# Retrieve the specified batch directly
X, y = train_dataloader_list[train_batch_num]
X, y = X.to(device), y.to(device)

# Make predictions using the model
pred = model(X)

# Move predictions and actual values back to CPU for plotting
pred = pred.cpu().detach().numpy()
y = y.cpu().detach().numpy()

# Plot the entire train data for the specified batch across all columns (angles)
plt.figure(figsize=(15, 10))

# Loop through each column (angle) and plot it
for col_id in range(y.shape[1]):
    plt.subplot(4, 2, col_id + 1)  # Create subplots for each column
    plt.plot(y[:, col_id], label='Actual Data', color='blue')
    plt.plot(pred[:, col_id], label='Predictions', color='red')
    plt.title(f'Column {col_id}: {train_dataset.columns[col_id]}')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()

plt.tight_layout()
plt.suptitle(f'Entire Train Data for Batch {train_batch_num}', y=1.02)
plt.show()


In [ ]:
# # Calculate the maximum batch number
# train_batch_num = len(train_dataloader) - 1
# print(f"train_batch_num: {train_batch_num}")
